In [1]:
!pip install pyproj

     |████████████████████████████████| 6.3 MB 5.4 MB/s 


In [3]:
from google.colab import drive, files
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# !unzip /content/drive/MyDrive/GSDS/2022-1/Big_Data_and_Knowledge/homework/dataset.zip -d /content/drive/MyDrive/GSDS/2022-1/Big_Data_and_Knowledge/homework/dataset

In [5]:
import pyproj
import warnings
import pandas as pd
import numpy as np
import os
from typing import List, Tuple, Dict
from bs4 import BeautifulSoup
from urllib.request import urlopen
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

PATH = "/content/drive/MyDrive/GSDS/2022-1/Big_Data_and_Knowledge/homework/dataset"
os.chdir(PATH)
print(os.getcwd())

/content/drive/MyDrive/GSDS/2022-1/Big_Data_and_Knowledge/homework/dataset


In [6]:
def read_csv(file_id:str, is_csv:bool=True, location:str=None, sheet_name:str=None, encoding:str='utf-8'):
    if is_csv:
        sheet_name = location.split('/')[-1]
    else:
        sheet_name = sheet_name
    url = f"https://docs.google.com/spreadsheets/d/{file_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    df = pd.read_csv(url, engine='python', delimiter=',', encoding=encoding)
    return df

In [7]:
def get_coord(system1:str, system2:str, init:tuple) -> tuple:
    proj1 = pyproj.Proj(init=system1)
    proj2 = pyproj.Proj(init=system2)
    y_coord, x_coord = pyproj.transform(proj1, proj2, init[0], init[1])
    return (x_coord, y_coord)

# 행정동 법정동 데이터

In [ ]:
legal_dict = dict()
with urlopen('https://www.seoul.go.kr/seoul/autonomy_sub.do') as url:
    doc = url.read()
soup = BeautifulSoup(doc, 'html.parser')
result = soup.find_all('div', class_='district tabcont', id=f"district4")
txt = result[0].get_text().replace('\r', '')
txt_list = [item for item in txt.split('\n') if len(item) > 0]

if txt_list[2] == "行 政 洞":
    txt_list.remove("行 政 洞")
    txt_list.remove("法 定 洞")
elif txt_list[2] == " 行 政 洞":
    txt_list.remove(" 行 政 洞")
    txt_list.remove(" 法 定 洞")

for item in txt_list:
    if item[:5] == ' '* 5 or '\t' in item or len(item) == 0:
        txt_list.remove(item)

print(txt_list)
print(len(txt_list))
print(txt_list[1])
# txt_list = txt_list[2:]
# for i, item in enumerate(txt_list):
#     print(i, item, len(item))
# for i in range(len(txt_list)-1):
#     if i % 2 == 0:
#         key = txt_list[i]
#         val = txt_list[i+1].split(', ')
#         legal_dict[key] = list(set(val))

# legal_dict

['16) 성동구(城東區) - 행정동(行政洞) 17,법정동(法定洞) 17', '성동구(城東區)', '왕십리제2동', '하왕십리동(下往十里洞)', '왕십리도선동', '상왕십리동(上往十里洞), 하왕십리동(下往十里洞), 도선동(道詵洞), 홍익동(弘益洞)', '마장동(馬場洞)', '마장동(馬場洞)', '사근동(沙斤洞)', '사근동(沙斤洞), 행당동(杏堂洞)', '행당제1동(杏堂第1洞)', '행당동(杏堂洞)', '행당제2동(杏堂第2洞)', '행당동(杏堂洞)', '응봉동(鷹峯洞)', '응봉동(鷹峯洞)', '금호1가동(金湖1街洞)', '금호동1가(金湖洞1街)', '금호2,3가동', '금호동2가(金湖洞2街), 금호동3가(金湖洞2街)', '금호4가동(金湖4街洞)', '금호동4가(金湖洞4街)', '옥수동(玉水洞)', '옥수동(玉水洞)', '성수1가제1동', '성수동1가(聖水洞1街)', '성수1가제2동', '성수동1가(聖水洞1街)', '성수2가제1동', '성수동2가(聖水洞2街)', '성수2가제3동', '성수동2가(聖水洞2街)', '송정동(松亭洞)', '송정동(松亭洞)', '용답동(龍踏洞)', '용답동(龍踏洞), 송정동(松亭洞)']
36
성동구(城東區)


In [ ]:
legal_dict = dict()
with urlopen('https://www.seoul.go.kr/seoul/autonomy_sub.do') as url:
    doc = url.read()
soup = BeautifulSoup(doc, 'html.parser')
for i in range(25):
    result = soup.find_all('div', class_='district tabcont', id=f"district{i+1}")
    txt = result[0].get_text().replace('\r', '')
    txt_list = [item for item in txt.split('\n') if len(item) > 0]

    if txt_list[2] == "行 政 洞":
        txt_list.remove("行 政 洞")
        txt_list.remove("法 定 洞")
    
    elif txt_list[2] == " 行 政 洞":
        txt_list.remove(" 行 政 洞")
        txt_list.remove(" 法 定 洞")

    for item in txt_list:
        if item[:5] == ' '* 5 or '\t' in item or len(item) == 0:
            txt_list.remove(item)

    district = txt_list[1]
    txt_list = txt_list[2:]
    for i in range(len(txt_list)):
        if i % 2 == 0:
            key = (district, txt_list[i])
            val = txt_list[i+1].split(', ')
            legal_dict[key] = list(set(val))

In [ ]:
print(legal_dict)

{('종로구(鐘路區)', '청운효자동'): ['옥인동(玉仁洞)', '신교동(新橋洞)', '창성동(昌成洞)', '청운동(淸雲洞)', '누상동(樓上洞)', '누하동(樓下洞)', '효자동(孝子洞)', '통인동(通仁洞)', '세종로(世宗路)', '궁정동(宮井洞)'], ('종로구(鐘路區)', '사직동(社稷洞)'): ['내수동(內需洞)', '신문로1.2가(新門路1.2街)', '필운동(弼雲洞)', '당주동(唐珠洞)', '세종로(世宗路)', '사직동(社稷洞)', '내자동(內資洞)', '체부동(體府洞)', '적선동(積善洞)', '통의동(通義洞)', '도렴동(都染洞)'], ('종로구(鐘路區)', '삼청동(三淸洞)'): ['화동(花洞)', '삼청동(三淸洞)', '사간동(司諫洞)', '안국동(安國洞)', '송현동(松峴洞)', '팔판동(八判洞)', '소격동(昭格洞)'], ('종로구(鐘路區)', '부암동(付岩洞)'): ['신영동(新營洞)', '부암동(付岩洞)', '홍지동(弘智洞)'], ('종로구(鐘路區)', '평창동(平倉洞)'): ['구기동(舊基洞)', '평창동(平倉洞)'], ('종로구(鐘路區)', '무악동(毋岳洞)'): ['무악동(毋岳洞)'], ('종로구(鐘路區)', '교남동(橋南洞)'): ['교남동(橋南洞)', '평동(平洞)', '행촌동(杏村洞)', '송월동(松月洞)', '교북동(橋北洞)', '홍파동(紅把洞)'], ('종로구(鐘路區)', '가회동(嘉會洞)'): ['원서동(苑西洞)', '가회동(嘉會洞)', '재동(齋洞)', '계동(桂洞)'], ('종로구(鐘路區)', '종로1.2.3.4가동'): ['와룡동(臥龍洞)', '운니동(雲泥洞)', '봉익동(鳳翼洞)', '원남동(苑南洞)', '서린동(瑞麟洞)', '청진동(淸進洞)', '종로2가(鐘路2街)', '낙원동(樂園洞)', '훈정동(薰井洞)', '수송동(壽松洞)', '인의동(仁義洞)', '관훈동(寬勳洞)', '중학동(中學洞)', '견지동(堅志洞)', '종로1가(鐘路1街)', '관수동(觀水洞)', '예지동(禮智洞)', '권농동(勸農洞)', 

In [ ]:
# 만리동1가 문제
string = '만리동1가(萬里洞1街)'
print(len(string))

idx = legal_dict[('중 구(中 區)', '중림동(中林洞)')].index('만리동1가(萬里洞1街)만리동2가(萬里洞2街)')
manli = legal_dict[('중 구(中 區)', '중림동(中林洞)')].pop(idx)
manli1, manli2 = manli[:len(string)], manli[len(string):]
legal_dict[('중 구(中 區)', '중림동(中林洞)')].append(manli1)
legal_dict[('중 구(中 區)', '중림동(中林洞)')].append(manli2)

print(legal_dict[('중 구(中 區)', '중림동(中林洞)')])

12
['의주로2가(義州路2街)', '중림동(中林洞)', '만리동1가(萬里洞1街)', '만리동2가(萬里洞2街)']


In [ ]:
## convert legal_dict to df
dong = pd.DataFrame(columns=['district', 'admin', 'legal'])
for key, val in legal_dict.items():
    dist, admin = key
    for legal in val:
        dong = dong.append({'district':dist.split('(')[0].replace(' ', ''), 
                            'admin':admin.split('(')[0], 
                            'legal':legal.split('(')[0]}, ignore_index=True)

dong.sort_values(by=['district', 'admin'], inplace=True)
dong.reset_index(drop=True, inplace=True)
dong.head()

,district,admin,legal
0,강남구,개포제1동,개포동
1,강남구,개포제2동,개포동
2,강남구,개포제4동,개포동
3,강남구,논현제1동,논현동
4,강남구,논현제2동,논현동


In [ ]:
## convert legal_dict to df
dong2 = pd.DataFrame(columns=['district', 'admin', 'legal'])
for key, val in legal_dict.items():
    dist, admin = key
    dong2 = dong2.append({'district':dist.split('(')[0].replace(' ', ''), 
                          'admin':admin.split('(')[0], 
                          'legal':[item.split('(')[0] for item in val]}, ignore_index=True)
        

dong2.sort_values(by=['district', 'admin'], inplace=True)
dong2.reset_index(drop=True, inplace=True)
dong2.head()

,district,admin,legal
0,강남구,개포제1동,[개포동]
1,강남구,개포제2동,[개포동]
2,강남구,개포제4동,[개포동]
3,강남구,논현제1동,[논현동]
4,강남구,논현제2동,[논현동]


In [ ]:
dong2[dong2['district'] == '중구']

,district,admin,legal
395,중구,광희동,"[광희동1가, 오장동, 광희동2가, 인현동, 충무로5가, 쌍림동, 예관동, 을지로7..."
396,중구,다산동,[신당동]
397,중구,동화동,[신당동]
398,중구,명동,"[삼각동, 명동2가, 남대문로1가, 회현동3가, 다동, 을지로1가, 회현동2가, 충..."
399,중구,소공동,"[태평로2가, 북창동, 순화동, 남대문로2가, 의주로1가, 남대문로3가, 남대문로4..."
400,중구,신당5동,[신당동]
401,중구,신당동,"[ 무학동, 신당동, 흥인동]"
402,중구,약수동,[신당동]
403,중구,을지로동,"[방산동, 주교동, 산림동, 저동2가, 을지로3가, 인현동, 을지로5가, 입정동, ..."
404,중구,장충동,"[묵정동, 장충동2가, 장충동1가]"


In [ ]:
file_id = "1C8HhTpudSvne46mPaM2JXX_EA6h9LVVy"
sheet_name = "sheet1"
coord = read_csv(file_id=file_id, is_csv=False, sheet_name=sheet_name)
print(coord.shape)
coord.head(1)

(467, 7)


코드 1111051500 1111053000 1111054000 1111055000 1111056000 1111057000 1111058000 1111060000 1111061500 1111063000 1111064000 1111065000 1111067000 1111068000 1111069000 1111070000 1111071000 1114052000 1114054000 1114055000 1114057000 1114058000 1114059000 1114060500 1114061500 1114062500 1114063500 1114064500 1114065000 1114066500 1114067000 1114068000 1117051000 1117052000 1117053000 1117055500 1117056000 1117057000 1117058000 1117059000 1117062500 1117063000 1117064000 1117065000 1117066000 1117068500 1117069000 1117070000 1120052000 1120053500 1120054000 1120055000 1120056000 1120057000 1120058000 1120059000 1120061500 1120062000 1120064500 1120065000 1120066000 1120067000 1120069000 1120072000 1120079000 1121571000 1121573000 1121574000 1121575000 1121576000 1121577000 1121578000 1121581000 1121582000 1121583000 1121584000 1121584700 1121585000 1121586000 1121587000 1123053600 1123054500 1123056000 1123057000 1123060000 1123061000 1123065000 1123066000 1123070500 1123071000 1123072000 1123073000 1123074000 1123075000 1126052000 1126054000 1126055000 1126056500 1126057000 1126057500 1126058000 1126059000 1126060000 1126061000 1126062000 1126063000 1126065500 1126066000 1126068000 1126069000 1129052500 1129055500 1129057500 1129058000 1129059000 1129060000 1129061000 1129062000 1129063000 1129064000 1129065000 1129066000 1129068500 1129070500 1129071500 1129072500 1129076000 1129077000 1129078000 1129081000 1130553400 1130553500 1130554500 1130555500 1130557500 1130559500 1130560300 1130560800 1130561500 1130562500 1130563500 1130564500 1130566000 1132051100 1132051200 1132051300 1132051400 1132051500 1132052100 1132052200 1132066000 1132067000 1132068000 1132068100 1132069000 1132070000 1132071000 1135056000 1135057000 1135058000 1135059500 1135060000 1135061100 1135061200 1135061900 1135062100 1135062400 1135062500 1135063000 1135064000 1135066500 1135067000 1135069500 1135070000 1135071000 1135072000 1138051000 1138052000 1138053000 1138055100 1138055200 1138056000 1138057000 1138058000 1138059000 1138060000 1138062500 1138063100 1138063200 1138064000 1138065000 1138069000 1141052000 1141055500 1141056500 1141058500 1141061500 1141062000 1141064000 1141065500 1141066000 1141068500 1141069000 1141070000 1141071000 1141072000 1144055500 1144056500 1144058500 1144059000 1144060000 1144061000 1144063000 1144065500 1144066000 1144068000 1144069000 1144070000 1144071000 1144072000 1144073000 1144074000 1147051000 1147052000 1147053000 1147054000 1147055000 1147056000 1147057000 1147058000 1147059000 1147060000 1147061000 1147061100 1147062000 1147063000 1147064000 1147065000 1147067000 1147068000 1150051000 1150052000 1150053000 1150053500 1150054000 1150055000 1150056000 1150057000 1150059000 1150059100 1150059300 1150060300 1150060400 1150060500 1150061100 1150061500 1150062000 1150063000 1150064000 1150064100 1153051000 1153052000 1153053000 1153054000 1153055000 1153056000 1153059500 1153072000 1153073000 1153074000 1153075000 1153076000 1153077000 1153078000 1153079000 1154551000 1154561000 1154562000 1154563000 1154564000 1154567000 1154568000 1154569000 1154570000 1154571000 1156051500 1156053500 1156054000 1156055000 1156056000 1156058500 1156060500 1156061000 1156062000 1156063000 1156065000 1156066000 1156067000 1156068000 1156069000 1156070000 1156071000 1156072000 1159051000 1159052000 1159053000 1159054000 1159055000 1159056000 1159060500 1159062000 1159063000 1159064000 1159065000 1159065100 1159066000 1159067000 1159068000 1162052500 1162054500 1162056500 1162057500 1162058500 1162059500 1162060500 1162061500 1162062500 1162063000 1162064500 1162065500 1162066500 1162068500 1162069500 1162071500 1162072500 1162073500 1162074500 1162076500 1162077500 1165051000 1165052000 1165053000 1165053100 1165054000 1165055000 1165056000 1165057000 1165058000 1165058100 1165059000 1165060000 1165061000 1165062000 1165062100 1165065100 1165065200 1165066000 1168051000 1168052100 1168053100 1168054500 1168056500 1168058000 1168

In [ ]:
data = coord.columns.values
print(data)
print(type(data))
print(data.shape)

['코드 1111051500 1111053000 1111054000 1111055000 1111056000 1111057000 1111058000 1111060000 1111061500 1111063000 1111064000 1111065000 1111067000 1111068000 1111069000 1111070000 1111071000 1114052000 1114054000 1114055000 1114057000 1114058000 1114059000 1114060500 1114061500 1114062500 1114063500 1114064500 1114065000 1114066500 1114067000 1114068000 1117051000 1117052000 1117053000 1117055500 1117056000 1117057000 1117058000 1117059000 1117062500 1117063000 1117064000 1117065000 1117066000 1117068500 1117069000 1117070000 1120052000 1120053500 1120054000 1120055000 1120056000 1120057000 1120058000 1120059000 1120061500 1120062000 1120064500 1120065000 1120066000 1120067000 1120069000 1120072000 1120079000 1121571000 1121573000 1121574000 1121575000 1121576000 1121577000 1121578000 1121581000 1121582000 1121583000 1121584000 1121584700 1121585000 1121586000 1121587000 1123053600 1123054500 1123056000 1123057000 1123060000 1123061000 1123065000 1123066000 1123070500 1123071000 11230

In [ ]:
hang = pd.DataFrame(columns=['code', 'city', 'district', 'admin', 'latitude', 'longitude'])
for i in range(data.shape[0]):
    if i == 4:
        continue
    item_list = data[i].split(' ')
    if i < 4:
        col = hang.columns.values[i]
    else:
        col = hang.columns.values[i-1]
    hang[col] = item_list[1:]

hang.drop('city', axis=1, inplace=True)
hang.head()

,code,district,admin,latitude,longitude
0,1111051500,종로구,청운효자동,37.584009,126.970626
1,1111053000,종로구,사직동,37.575408,126.965944
2,1111054000,종로구,삼청동,37.590758,126.980996
3,1111055000,종로구,부암동,37.594768,126.965574
4,1111056000,종로구,평창동,37.613029,126.974485


In [ ]:
hang['code'].iloc[0]

'1111051500'

In [ ]:
for i in range(hang.shape[0]):
    code = hang['code'].iloc[i]
    hang['code'].iloc[i] = code[:-2]

In [ ]:
hang.head()

,code,district,admin,latitude,longitude
0,11110515,종로구,청운효자동,37.584009,126.970626
1,11110530,종로구,사직동,37.575408,126.965944
2,11110540,종로구,삼청동,37.590758,126.980996
3,11110550,종로구,부암동,37.594768,126.965574
4,11110560,종로구,평창동,37.613029,126.974485


In [ ]:
hang.to_csv(f"{PATH}/admin.csv", index=False)

# 유동인구

In [8]:
os.listdir()

['서울특별시 코로나19 자치구별 확진자 발생동향.csv',
 '서울시 우리마을가게 상권분석서비스(상권-생활인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권_상주인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권-상권변화지표).csv',
 '서울시 우리마을가게 상권분석서비스(상권-직장인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권-아파트).csv',
 '서울시 우리마을가게 상권분석서비스(상권-추정매출).csv',
 '서울시 우리마을가게 상권분석서비스(상권-점포).csv',
 '서울시 우리마을가게 상권분석서비스(상권영역).csv',
 '서울특별시 건축물대장 공통 새주소코드 정보.csv',
 'seoul_dong.csv',
 'seoul_dong.txt',
 '__MACOSX',
 'sangju.csv',
 'admin.csv',
 'services.csv',
 'change_codes.csv',
 'unique_md.csv',
 'md_class_code.csv',
 'change.csv',
 'calendar.csv',
 'md_region.csv',
 'working_pop.csv',
 'gender_sales.csv',
 'weekday_sales.csv',
 'covid_cases.csv',
 'md_info.gsheet']

In [9]:
## 상주인구, 직장인구 파악
location = f"{PATH}/{os.listdir()[1]}"
sangju = pd.read_csv(location, engine='python', encoding='cp949')
sangju.columns.values

array(['기준 년코드', '기준_분기_코드', '상권_구분_코드', ' 상권_구분_코드_명', '상권_코드',
       '상권_코드_명', '총_생활인구_수', '남성_생활인구_수', '여성_생활인구_수', '연령대_10_생활인구_수',
       '연령대_20_생활인구_수', '연령대_30_생활인구_수', '연령대_40_생활인구_수', '연령대_50_생활인구_수',
       '연령대_60_이상_생활인구_수', '시간대_1_생활인구_수', '시간대_2_생활인구_수', '시간대_3_생활인구_수',
       '시간대_4_생활인구_수', '시간대_5_생활인구_수', '시간대_6_생활인구_수', '월요일_생활인구_수',
       '화요일_생활인구_수', '수요일_생활인구_수', '목요일_생활인구_수', '금요일_생활인구_수',
       '토요일_생활인구_수', '일요일_생활인구_수', '남성연령대_10_월요일시간대_1_생활인구_수',
       '남성연령대_10_월요일시간대_2_생활인구_수', '남성연령대_10_월요일시간대_3_생활인구_수',
       '남성연령대_10_월요일시간대_4_생활인구_수', '남성연령대_10_월요일시간대_5_생활인구_수',
       '남성연령대_10_월요일시간대_6_생활인구_수', '남성연령대_10_화요일시간대_1_생활인구_수',
       '남성연령대_10_화요일시간대_2_생활인구_수', '남성연령대_10_화요일시간대_3_생활인구_수',
       '남성연령대_10_화요일시간대_4_생활인구_수', '남성연령대_10_화요일시간대_5_생활인구_수',
       '남성연령대_10_화요일시간대_6_생활인구_수', '남성연령대_10_수요일시간대_1_생활인구_수',
       '남성연령대_10_수요일시간대_2_생활인구_수', '남성연령대_10_수요일시간대_3_생활인구_수',
       '남성연령대_10_수요일시간대_4_생활인구_수', '남성연령대_10_수요일시간대_5_생활인구_수',
       '남성연령대

In [10]:
sangju.head()

,기준 년코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수,남성_생활인구_수,여성_생활인구_수,연령대_10_생활인구_수,연령대_20_생활인구_수,연령대_30_생활인구_수,연령대_40_생활인구_수,연령대_50_생활인구_수,연령대_60_이상_생활인구_수,시간대_1_생활인구_수,시간대_2_생활인구_수,시간대_3_생활인구_수,시간대_4_생활인구_수,시간대_5_생활인구_수,시간대_6_생활인구_수,월요일_생활인구_수,화요일_생활인구_수,수요일_생활인구_수,목요일_생활인구_수,금요일_생활인구_수,토요일_생활인구_수,일요일_생활인구_수,남성연령대_10_월요일시간대_1_생활인구_수,남성연령대_10_월요일시간대_2_생활인구_수,남성연령대_10_월요일시간대_3_생활인구_수,남성연령대_10_월요일시간대_4_생활인구_수,남성연령대_10_월요일시간대_5_생활인구_수,남성연령대_10_월요일시간대_6_생활인구_수,남성연령대_10_화요일시간대_1_생활인구_수,남성연령대_10_화요일시간대_2_생활인구_수,남성연령대_10_화요일시간대_3_생활인구_수,남성연령대_10_화요일시간대_4_생활인구_수,남성연령대_10_화요일시간대_5_생활인구_수,남성연령대_10_화요일시간대_6_생활인구_수,남성연령대_10_수요일시간대_1_생활인구_수,남성연령대_10_수요일시간대_2_생활인구_수,남성연령대_10_수요일시간대_3_생활인구_수,남성연령대_10_수요일시간대_4_생활인구_수,남성연령대_10_수요일시간대_5_생활인구_수,남성연령대_10_수요일시간대_6_생활인구_수,남성연령대_10_목요일시간대_1_생활인구_수,남성연령대_10_목요일시간대_2_생활인구_수,남성연령대_10_목요일시간대_3_생활인구_수,남성연령대_10_목요일시간대_4_생활인구_수,남성연령대_10_목요일시간대_5_생활인구_수,남성연령대_10_목요일시간대_6_생활인구_수,남성연령대_10_금요일시간대_1_생활인구_수,남성연령대_10_금요일시간대_2_생활인구_수,남성연령대_10_금요일시간대_3_생활인구_수,남성연령대_10_금요일시간대_4_생활인구_수,남성연령대_10_금요일시간대_5_생활인구_수,남성연령대_10_금요일시간대_6_생활인구_수,남성연령대_10_토요일시간대_1_생활인구_수,남성연령대_10_토요일시간대_2_생활인구_수,남성연령대_10_토요일시간대_3_생활인구_수,남성연령대_10_토요일시간대_4_생활인구_수,남성연령대_10_토요일시간대_5_생활인구_수,남성연령대_10_토요일시간대_6_생활인구_수,남성연령대_10_일요일시간대_1_생활인구_수,남성연령대_10_일요일시간대_2_생활인구_수,남성연령대_10_일요일시간대_3_생활인구_수,남성연령대_10_일요일시간대_4_생활인구_수,남성연령대_10_일요일시간대_5_생활인구_수,남성연령대_10_일요일시간대_6_생활인구_수,남성연령대_20_월요일시간대_1_생활인구_수,남성연령대_20_월요일시간대_2_생활인구_수,남성연령대_20_월요일시간대_3_생활인구_수,남성연령대_20_월요일시간대_4_생활인구_수,남성연령대_20_월요일시간대_5_생활인구_수,남성연령대_20_월요일시간대_6_생활인구_수,남성연령대_20_화요일시간대_1_생활인구_수,남성연령대_20_화요일시간대_2_생활인구_수,남성연령대_20_화요일시간대_3_생활인구_수,남성연령대_20_화요일시간대_4_생활인구_수,남성연령대_20_화요일시간대_5_생활인구_수,남성연령대_20_화요일시간대_6_생활인구_수,남성연령대_20_수요일시간대_1_생활인구_수,남성연령대_20_수요일시간대_2_생활인구_수,남성연령대_20_수요일시간대_3_생활인구_수,남성연령대_20_수요일시간대_4_생활인구_수,남성연령대_20_수요일시간대_5_생활인구_수,남성연령대_20_수요일시간대_6_생활인구_수,남성연령대_20_목요일시간대_1_생활인구_수,남성연령대_20_목요일시간대_2_생활인구_수,남성연령대_20_목요일시간대_3_생활인구_수,남성연령대_20_목요일시간대_4_생활인구_수,남성연령대_20_목요일시간대_5_생활인구_수,남성연령대_20_목요일시간대_6_생활인구_수,남성연령대_20_금요일시간대_1_생활인구_수,남성연령대_20_금요일시간대_2_생활인구_수,남성연령대_20_금요일시간대_3_생활인구_수,남성연령대_20_금요일시간대_4_생활인구_수,남성연령대_20_금요일시간대_5_생활인구_수,남성연령대_20_금요일시간대_6_생활인구_수,남성연령대_20_토요일시간대_1_생활인구_수,남성연령대_20_토요일시간대_2_생활인구_수,남성연령대_20_토요일시간대_3_생활인구_수,남성연령대_20_토요일시간대_4_생활인구_수,남성연령대_20_토요일시간대_5_생활인구_수,남성연령대_20_토요일시간대_6_생활인구_수,남성연령대_20_일요일시간대_1_생활인구_수,남성연령대_20_일요일시간대_2_생활인구_수,남성연령대_20_일요일시간대_3_생활인구_수,남성연령대_20_일요일시간대_4_생활인구_수,남성연령대_20_일요일시간대_5_생활인구_수,남성연령대_20_일요일시간대_6_생활인구_수,남성연령대_30_월요일시간대_1_생활인구_수,남성연령대_30_월요일시간대_2_생활인구_수,남성연령대_30_월요일시간대_3_생활인구_수,남성연령대_30_월요일시간대_4_생활인구_수,남성연령대_30_월요일시간대_5_생활인구_수,남성연령대_30_월요일시간대_6_생활인구_수,남성연령대_30_화요일시간대_1_생활인구_수,남성연령대_30_화요일시간대_2_생활인구_수,남성연령대_30_화요일시간대_3_생활인구_수,남성연령대_30_화요일시간대_4_생활인구_수,남성연령대_30_화요일시간대_5_생활인구_수,남성연령대_30_화요일시간대_6_생활인구_수,남성연령대_30_수요일시간대_1_생활인구_수,남성연령대_30_수요일시간대_2_생활인구_수,남성연령대_30_수요일시간대_3_생활인구_수,남성연령대_30_수요일시간대_4_생활인구_수,남성연령대_30_수요일시간대_5_생활인구_수,남성연령대_30_수요일시간대_6_생활인구_수,남성연령대_30_목요일시간대_1_생활인구_수,남성연령대_30_목요일시간대_2_생활인구_수,남성연령대_30_목요일시간대_3_생활인구_수,남성연령대_30_목요일시간대_4_생활인구_수,남성연령대_30_목요일시간대_5_생활인구_수,남성연령대_30_목요일시간대_6_생활인구_수,남성연령대_30_금요일시간대_1_생활인구_수,남성연령대_30_금요일시간대_2_생활인구_수,남성연령대_30_금요일시간대_3_생활인구_수,남성연령대_30_금요일시간대_4_생활인구_수,남성연령대_30_금요일시간대_5_생활인구_수,남성연령대_30_금요일시간대_6_생활인구_수,남성연령대_30_토요일시간대_1_생활인구_수,남성연령대_30_토요일시간대_2_생활인구_수,남성연령대_30_토요일시간대_3_생활인구_수,남성연령대_30_토요일시간대_4_생활인구_수,남성연령대_30_토요일시간대_5_생활인구_수,남성연령대_30_토요일시간대_6_생활인구_수,남성연령대_30_일요일시간대_1_생활인구_수,남성연령대_30_일요일시간대_2_생활인구_수,남성연령대_30_일요일시간대_3_생활인구_수,남성연령대_30_일요일시간대_4_생활인구_수,남성연령대_30_일요일시간대_5_생활인구_수,남성연령대_30_일요일시간대_6_생활인구_수,남성연령대_40_월요일시간대_1_생활인구_수,남성연령대_40_월요일시간대_2_생활인구_수,남성연령대_40_월요일시간대_3_생활인구_수,남성연령대_40_월요일시간대_4_생활인구_수,남성연령대_40_월요일시간대_5_생활인구_수,남성연령대_40_월요일시간대_6_생활인구_수,남성연령대_40_화요일시간대_1_생활인구_수,남성연령대_40_화요일시간대_2_생활인구_수,남성연령대_40_화요일시간대_3_생활인구_수,남성연령대_40_화요일시간대_4_생활인구_수,남성연령대_40_화요일시간대_5_생활인구_수,남성연령대_40_화요일시간대_6_생활인구_수,남성연령대_40_수요일시간대_1_생활인구_수,남성연령대_40_수요일시간대_2_생활인구_수,남성연령대_40_수요일시간대_3_생활인구_수,남성연령대_40_수요일시간대_4_생활인구_수,남성연령대_40_수요일시간대_5_생활인구_수,남성연령대_40_수요일시간대_6_생활인구_수,남성연령대_40_목요일시간대_1_생활인구_수,남성연령대_40_목요일시간대_2_생활인구_수,남성연령대_40_목요일시간대_3_생활인구_수,남성연

In [43]:
sangju[sangju['상권_코드'] == 1001231][['기준 년코드', '총_생활인구_수']]

,기준 년코드,총_생활인구_수
264,2021,160449
1759,2021,163141
3254,2021,265945
4749,2021,248369
6244,2020,295498
7739,2020,310143
9234,2020,287954
10729,2020,276545
12224,2019,302692
13719,2019,285910


In [27]:
pop_male = pd.DataFrame()
pop_male['year'] = sangju['기준 년코드']
pop_male['quarter'] = sangju['기준_분기_코드']
pop_male['md_class_code'] = sangju['상권_구분_코드']
pop_male['md_code'] = sangju['상권_코드']
pop_male['gender'] = '남성'

pop_female = pd.DataFrame()
pop_female['year'] = sangju['기준 년코드']
pop_female['quarter'] = sangju['기준_분기_코드']
pop_female['md_class_code'] = sangju['상권_구분_코드']
pop_female['md_code'] = sangju['상권_코드']
pop_female['gender'] = '여성'

pop = pd.concat([pop_male, pop_female], axis=0)
print(pop_male.shape)
print(pop_female.shape)
print(pop.shape)
pop.sort_values(by=['year', 'quarter', 'md_code'], ascending=[False, False, False], inplace=True)
pop.reset_index(drop=True, inplace=True)
pop.head()

(47758, 5)
(47758, 5)
(95516, 5)


,year,quarter,md_class_code,md_code,gender
0,2021,4,U,1001496,남성
1,2021,4,U,1001496,여성
2,2021,4,U,1001495,남성
3,2021,4,U,1001495,여성
4,2021,4,U,1001494,남성


In [12]:
age_dfs = []
for i in range(6):
    age = (i+1) * 10
    age_df = pd.DataFrame()
    age_df['year'] = sangju['기준_년_코드']
    age_df['quarter'] = sangju['기준_분기_코드']
    age_df['mdcode'] = sangju['상권 코드']
    age_df['age'] = age
    age_dfs.append(age_df)

ages = pd.concat(age_dfs, axis=0)
print(age_dfs[0].shape)
print(ages.shape)
ages.sort_values(by=['year', 'quarter', 'mdcode', 'age'], ascending=[False, False, False, True], inplace=True)
ages.reset_index(drop=True, inplace=True)
ages.head()

(45742, 4)
(274452, 4)


,year,quarter,mdcode,age
0,2021,4,1001496,10
1,2021,4,1001496,20
2,2021,4,1001496,30
3,2021,4,1001496,40
4,2021,4,1001496,50


In [13]:
sexage = pd.merge(left=pop, right=ages, on=['year', 'quarter', 'mdcode'], how='left')
print(sexage.shape)
sexage.head(15)

(548904, 5)


,year,quarter,mdcode,gender,age
0,2021,4,1001496,남성,10
1,2021,4,1001496,남성,20
2,2021,4,1001496,남성,30
3,2021,4,1001496,남성,40
4,2021,4,1001496,남성,50
5,2021,4,1001496,남성,60
6,2021,4,1001496,여성,10
7,2021,4,1001496,여성,20
8,2021,4,1001496,여성,30
9,2021,4,1001496,여성,40


In [ ]:
sexage.isnull().sum()

In [28]:
pop['population'] = np.zeros(shape=(pop.shape[0]))

In [31]:
pop['population'] = np.zeros(shape=(pop.shape[0]))
for i in range(pop.shape[0]):
    row = pop.iloc[i]
    year, qtr, code = row['year'], row['quarter'], row['md_code']
    gender = row['gender']
    subdf = sangju[(sangju['기준 년코드'] == year) & (sangju['기준_분기_코드'] == qtr) & (sangju['상권_코드'] == code)]
    for col in subdf.columns.values:
        if gender+'_' in col:
            # print(col)
            pop['population'].iloc[i] = subdf[col].iloc[0]

In [32]:
pop.head()

,year,quarter,md_class_code,md_code,gender,population
0,2021,4,U,1001496,남성,51045.0
1,2021,4,U,1001496,여성,51249.0
2,2021,4,U,1001495,남성,1642782.0
3,2021,4,U,1001495,여성,1728551.0
4,2021,4,U,1001494,남성,2102996.0


In [33]:
pop.to_csv(f"{PATH}/floating_pop.csv", index=False, encoding='utf8')

In [ ]:
# sexage.isnull().sum()

In [ ]:
# location = f"{PATH}/{os.listdir()[4]}"
# work = pd.read_csv(location, engine='python', encoding='cp949')
# work.head()

In [ ]:
# work_male = pd.DataFrame()
# work_male['year'] = work['기준_년월_코드']
# work_male['quarter'] = work['기준_분기_코드']
# work_male['mdcode'] = work['상권_코드']
# work_male['gender'] = '남성'

# work_female = pd.DataFrame()
# work_female['year'] = work['기준_년월_코드']
# work_female['quarter'] = work['기준_분기_코드']
# work_female['mdcode'] = work['상권_코드']
# work_female['gender'] = '여성'

# workpop = pd.concat([work_male, work_female], axis=0)
# print(work_male.shape)
# print(work_female.shape)
# print(workpop.shape)
# workpop.sort_values(by=['year', 'quarter', 'mdcode'], ascending=[False, False, False], inplace=True)
# workpop.reset_index(drop=True, inplace=True)
# workpop.head()

In [ ]:
# work_age_dfs = []
# for i in range(6):
#     age = (i+1) * 10
#     age_df = pd.DataFrame()
#     age_df['year'] = work['기준_년월_코드']
#     age_df['quarter'] = work['기준_분기_코드']
#     age_df['mdcode'] = work['상권_코드']
#     age_df['age'] = str(age)
#     work_age_dfs.append(age_df)

# work_ages = pd.concat(work_age_dfs, axis=0)
# print(work_age_dfs[0].shape)
# print(work_ages.shape)
# work_ages.sort_values(by=['year', 'quarter', 'mdcode', 'age'], ascending=[False, False, False, True], inplace=True)
# work_ages.reset_index(drop=True, inplace=True)
# work_ages.head()

In [ ]:
# work_sexage = pd.merge(left=workpop, right=work_ages, on=['year', 'quarter', 'mdcode'], how='left')
# print(work_sexage.shape)
# work_sexage.head(15)

In [ ]:
# null = work_sexage.isnull()
# null[null['age'] == True]

In [ ]:
# work_sexage['population'] = np.zeros(shape=(work_sexage.shape[0]))
# for i in range(work_sexage.shape[0]):
#     row = work_sexage.iloc[i]
#     year, qtr, code = row['year'], row['quarter'], row['mdcode']
#     gender, age = row['gender'], str(row['age'])
#     subdf = work[(work['기준_년월_코드'] == year) & (work['기준_분기_코드'] == qtr) & (work['상권_코드'] == code)]
#     for col in subdf.columns.values:
#         if gender in col and age in col:
#             work_sexage['population'].iloc[i] = subdf[col].iloc[0]

# work_sexage.head()

In [ ]:
# sexage.to_csv(f"{PATH}/sangju.csv", index=False)
# work_sexage.to_csv(f"{PATH}/working_pop.csv", index=False)

# Calendar

In [ ]:
from datetime import date, timedelta

calendar = pd.DataFrame()
init = date(2010, 1, 1)
dates = []
while init < date(2023, 1, 1):
    dates.append(init)
    init += timedelta(days=1)

calendar['date'] = dates
calendar.head()

,date
0,2010-01-01
1,2010-01-02
2,2010-01-03
3,2010-01-04
4,2010-01-05


In [ ]:
print(calendar['date'].iloc[0])
print(type(calendar['date'].iloc[0]))
calendar.tail()

2010-01-01
<class 'datetime.date'>


,date
4743,2022-12-27
4744,2022-12-28
4745,2022-12-29
4746,2022-12-30
4747,2022-12-31


In [ ]:
calendar['date'] = pd.to_datetime(calendar['date'])
calendar['year'] = calendar['date'].dt.year
calendar['month'] = calendar['date'].dt.month
calendar['day'] = calendar['date'].dt.day
calendar['weekday'] = calendar['date'].dt.dayofweek # Monday = 0, Sunday = 6
calendar.head()

,date,year,month,day,weekday
0,2010-01-01,2010,1,1,4
1,2010-01-02,2010,1,2,5
2,2010-01-03,2010,1,3,6
3,2010-01-04,2010,1,4,0
4,2010-01-05,2010,1,5,1


In [ ]:
def quarter(x):
    if 1 <= x <= 3:
        return 1
    elif 4 <= x <= 6:
        return 2
    elif 7 <= x<= 9:
        return 3
    else:
        return 4


def season(x):
    if 3 <= x <= 5:
        return 1
    elif 6 <= x <= 8:
        return 2
    elif 9 <= x <= 11:
        return 3
    else:
        return 4

In [ ]:
calendar['quarter'] = calendar['month'].map(quarter)
calendar['season'] = calendar['month'].map(season)
calendar.head()

,date,year,month,day,weekday,quarter,season
0,2010-01-01,2010,1,1,4,1,4
1,2010-01-02,2010,1,2,5,1,4
2,2010-01-03,2010,1,3,6,1,4
3,2010-01-04,2010,1,4,0,1,4
4,2010-01-05,2010,1,5,1,1,4


## 특일 정보

In [ ]:
import requests
from urllib import parse

def get_holiday(url:str, key:str, year:int) -> pd.DataFrame:
    holiday = pd.DataFrame(columns=['date', 'isholiday'])

    params = {
        'ServiceKey': key,
        'solYear': year,
        'numOfRows': 100
    }

    response = requests.get(url, params=params)
    xml_soup = BeautifulSoup(response.text, "lxml")
    items = xml_soup.find_all("item")

    for item in items:
        isholiday = item.find("isholiday").get_text().strip()
        date = item.find("locdate").get_text().strip()
        holiday = holiday.append({
            'date': date,
            'isholiday': isholiday
        }, ignore_index=True)

    return holiday

In [ ]:
url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService"
utf_key = "q2VhG7okUhr%2F3AjcCzOBLYeyJj2nHr1F7wJEVI6DuuO83eJ1DvygulMfDfRtzsGRV7JwViHrO%2Bg49DdSQNkKFQ%3D%3D"
decode_key = parse.unquote(utf_key)
url_holiday = url + "/getRestDeInfo"

In [ ]:
years = [i for i in range(2010, 2023)]
holiday_dfs = []
for year in years:
    holdf = get_holiday(url_holiday, decode_key, year)
    holiday_dfs.append(holdf)

print(len(holiday_dfs))
holiday_dfs[0].head()

13


,date,isholiday
0,20100101,Y
1,20100213,Y
2,20100214,Y
3,20100215,Y
4,20100301,Y


In [ ]:
holiday = pd.DataFrame()
for df in holiday_dfs:
    holiday = pd.concat([holiday, df], axis=0)
holiday.reset_index(drop=True, inplace=True)
holiday['date'] = pd.to_datetime(holiday['date'])
holiday.head()

,date,isholiday
0,2010-01-01,Y
1,2010-02-13,Y
2,2010-02-14,Y
3,2010-02-15,Y
4,2010-03-01,Y


In [ ]:
print(type(holiday['date'].iloc[0]))
print(type(calendar['date'].iloc[0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [ ]:
calmerge = pd.merge(left=calendar, right=holiday, on='date', how='left')
calmerge.head()

,date,year,month,day,weekday,quarter,season,isholiday
0,2010-01-01,2010,1,1,4,1,4,Y
1,2010-01-02,2010,1,2,5,1,4,NaN
2,2010-01-03,2010,1,3,6,1,4,NaN
3,2010-01-04,2010,1,4,0,1,4,NaN
4,2010-01-05,2010,1,5,1,1,4,NaN


In [ ]:
calmerge['isholiday'].fillna('N', inplace=True)
calmerge.head()

,date,year,month,day,weekday,quarter,season,isholiday
0,2010-01-01,2010,1,1,4,1,4,Y
1,2010-01-02,2010,1,2,5,1,4,N
2,2010-01-03,2010,1,3,6,1,4,N
3,2010-01-04,2010,1,4,0,1,4,N
4,2010-01-05,2010,1,5,1,1,4,N


In [ ]:
def is_holiday(x):
    if x['weekday'] >= 5 or x['isholiday'] == 1:
        return 1
    return 0

calmerge['holiday'] = calmerge[['weekday', 'isholiday']].apply(is_holiday, axis=1)

In [ ]:
calmerge.head()

,date,year,month,day,weekday,quarter,season,isholiday,holiday
0,2010-01-01,2010,1,1,4,1,4,1,1
1,2010-01-02,2010,1,2,5,1,4,0,1
2,2010-01-03,2010,1,3,6,1,4,0,1
3,2010-01-04,2010,1,4,0,1,4,0,0
4,2010-01-05,2010,1,5,1,1,4,0,0


In [ ]:
calmerge.columns = ['date', 'year', 'month', 'day', 'weekday', 'quarter', 'season', 'legal_holiday', 'agg_holiday']
calmerge.head()

,date,year,month,day,weekday,quarter,season,legal_holiday,agg_holiday
0,2010-01-01,2010,1,1,4,1,4,1,1
1,2010-01-02,2010,1,2,5,1,4,0,1
2,2010-01-03,2010,1,3,6,1,4,0,1
3,2010-01-04,2010,1,4,0,1,4,0,0
4,2010-01-05,2010,1,5,1,1,4,0,0


In [ ]:
print(calmerge.shape)
calmerge.drop_duplicates(subset=['date'], inplace=True)
print(calmerge.shape)

(4749, 9)
(4748, 9)


In [ ]:
calmerge.to_csv(f"{PATH}/calendar.csv", index=False)

# COVID

In [ ]:
os.listdir()

['서울특별시 코로나19 자치구별 확진자 발생동향.csv',
 '서울시 우리마을가게 상권분석서비스(상권-생활인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권_상주인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권-상권변화지표).csv',
 '서울시 우리마을가게 상권분석서비스(상권-직장인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권-아파트).csv',
 '서울시 우리마을가게 상권분석서비스(상권-추정매출).csv',
 '서울시 우리마을가게 상권분석서비스(상권-점포).csv',
 '서울시 우리마을가게 상권분석서비스(상권영역).csv',
 '서울특별시 건축물대장 공통 새주소코드 정보.csv',
 'seoul_dong.csv',
 'seoul_dong.txt',
 '__MACOSX',
 'sangju.csv',
 'change.csv',
 'working_pop.csv',
 'admin.csv',
 'covid_cases.csv',
 'md_region.csv',
 'services.csv',
 'unique_md.csv',
 'md_class_code.csv',
 'change_codes.csv',
 'calendar.csv',
 'gender_sales.csv',
 'weekday_sales.csv']

In [ ]:
hang.head()

NameError: ignored

In [ ]:
covid = pd.read_csv(os.listdir()[0], engine='python', encoding='cp949')
covid.columns.values

array(['자치구 기준일', '종로구 전체', '종로구 추가', '중구 전체', '중구 추가', '용산구 전체',
       '용산구 추가', '성동구 전체', '성동구 추가', '광진구 전체', '광진구 추가', '동대문구 전체',
       '동대문구 추가', '중랑구 전체', '중랑구 추가', '성북구 전체', '성북구 추가', '강북구 전체',
       '강북구 추가', '도봉구 전체', '도봉구 추가', '노원구 전체', '노원구 추가', '은평구 전체',
       '은평구 추가', '서대문구 전체', '서대문구 추가', '마포구 전체', '마포구 추가', '양천구 전체',
       '양천구 추가', '강서구 전체', '강서구 추가', '구로구 전체', '구로구 추가', '금천구 전체',
       '금천구 추가', '영등포구 전체', '영등포구 추가', '동작구 전체', '동작구 추가', '관악구 전체',
       '관악구 추가', '서초구 전체', '서추구 추가', '강남구 전체', '강남구 추가', '송파구 전체',
       '송파구 추가', '강동구 전체', '강동구 추가', '기타 전체', '기타 추가'], dtype=object)

In [ ]:
to_use = [covid.columns.values[0]] + [col for col in covid.columns.values if '추가' in col]
print(to_use)

['자치구 기준일', '종로구 추가', '중구 추가', '용산구 추가', '성동구 추가', '광진구 추가', '동대문구 추가', '중랑구 추가', '성북구 추가', '강북구 추가', '도봉구 추가', '노원구 추가', '은평구 추가', '서대문구 추가', '마포구 추가', '양천구 추가', '강서구 추가', '구로구 추가', '금천구 추가', '영등포구 추가', '동작구 추가', '관악구 추가', '서추구 추가', '강남구 추가', '송파구 추가', '강동구 추가', '기타 추가']


In [ ]:
covid_df = covid[to_use]
covid_df.head()

,자치구 기준일,종로구 추가,중구 추가,용산구 추가,성동구 추가,광진구 추가,동대문구 추가,중랑구 추가,성북구 추가,강북구 추가,도봉구 추가,노원구 추가,은평구 추가,서대문구 추가,마포구 추가,양천구 추가,강서구 추가,구로구 추가,금천구 추가,영등포구 추가,동작구 추가,관악구 추가,서추구 추가,강남구 추가,송파구 추가,강동구 추가,기타 추가
0,2022.03.22.00,1021,1094,1536,2231,1734,2814,3058,2176,2263,2187,3155,2002,2268,2811,2863,4034,2689,2003,2284,2735,2912,1994,3261,4723,2091,3094
1,2022.03.21.00,437,278,575,833,1819,1037,1250,1406,761,1201,3569,2640,1292,1237,2005,2012,1022,560,2486,1341,1694,2064,2834,2335,2378,2012
2,2022.03.20.00,1023,998,1370,2577,2334,2017,2396,3052,1854,2007,4210,2487,1801,2420,2730,4125,2841,1585,1922,2715,3433,2627,2198,4524,3090,3534
3,2022.03.19.00,1061,912,1740,2274,2655,2695,2930,3569,2027,2228,4846,3606,2177,2734,3689,3985,3356,1914,4661,3209,4678,2930,5317,4816,2955,5168
4,2022.03.18.00,1172,977,2045,2649,2982,3197,2820,3751,2248,2565,2898,3556,3283,2706,3722,6367,3122,1950,3703,3020,3066,2802,3892,5107,3628,4769


In [ ]:
new_cols = []
for col in covid_df.columns.values:
    if '추가' in col:
        col = col[:-3]
    if '자치구' in col:
        col = 'date'
    if col == '서추구':
        col = '서초구'
    new_cols.append(col)

covid_df.columns = new_cols
covid_df.head()

,date,종로구,중구,용산구,성동구,광진구,동대문구,중랑구,성북구,강북구,도봉구,노원구,은평구,서대문구,마포구,양천구,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구,기타
0,2022.03.22.00,1021,1094,1536,2231,1734,2814,3058,2176,2263,2187,3155,2002,2268,2811,2863,4034,2689,2003,2284,2735,2912,1994,3261,4723,2091,3094
1,2022.03.21.00,437,278,575,833,1819,1037,1250,1406,761,1201,3569,2640,1292,1237,2005,2012,1022,560,2486,1341,1694,2064,2834,2335,2378,2012
2,2022.03.20.00,1023,998,1370,2577,2334,2017,2396,3052,1854,2007,4210,2487,1801,2420,2730,4125,2841,1585,1922,2715,3433,2627,2198,4524,3090,3534
3,2022.03.19.00,1061,912,1740,2274,2655,2695,2930,3569,2027,2228,4846,3606,2177,2734,3689,3985,3356,1914,4661,3209,4678,2930,5317,4816,2955,5168
4,2022.03.18.00,1172,977,2045,2649,2982,3197,2820,3751,2248,2565,2898,3556,3283,2706,3722,6367,3122,1950,3703,3020,3066,2802,3892,5107,3628,4769


In [ ]:
covid_df['date'].unique()

array(['2022.03.22.00', '2022.03.21.00', '2022.03.20.00', '2022.03.19.00',
       '2022.03.18.00', '2022.03.17.00', '2022.03.16.00', '2022.03.15.00',
       '2022.03.14.00', '2022.03.13.00', '2022.03.12.00', '2022.03.11.00',
       '2022.03.10.00', '2022.03.09.00', '2022.03.08.00', '2022.03.07.00',
       '2022.03.06.00', '2022.03.05.00', '2022.03.04.00', '2022.03.03.00',
       '2022.03.02.00', '2022.03.01.00', '2022.02.28.00', '2022.02.27.00',
       '2022.02.26.00', '2022.02.25.00', '2022.02.24.00', '2022.02.23.00',
       '2022.02.22.00', '2022.02.21.00', '2022.02.20.00', '2022.02.19.00',
       '2022.02.18.00', '2022.02.17.00', '2022.02.16.00', '2022.02.15.00',
       '2022.02.14.00', '2022.02.13.00', '2022.02.12.00', '2022.02.11.00',
       '2022.02.10.00', '2022.02.09.00', '2022.02.08.00', '2022.02.07.00',
       '2022.02.06.00', '2022.02.05.00', '2022.02.04.00', '2022.02.03.00',
       '2022.02.02.00', '2022.02.01.00', '2022.01.31.00', '2022.01.30.00',
       '2022.01.29.00', '

In [ ]:
covid_df.tail()

,date,종로구,중구,용산구,성동구,광진구,동대문구,중랑구,성북구,강북구,도봉구,노원구,은평구,서대문구,마포구,양천구,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구,기타
773,20.02.09.10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
774,20.02.08.10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
775,20.02.07.10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
776,20.02.06.10,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
777,20.02.05.10,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
covid_df['date'] = covid_df['date'].map(lambda x: x[:-3])
covid_df.head()

,date,종로구,중구,용산구,성동구,광진구,동대문구,중랑구,성북구,강북구,도봉구,노원구,은평구,서대문구,마포구,양천구,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구,기타
0,2022.03.22,1021,1094,1536,2231,1734,2814,3058,2176,2263,2187,3155,2002,2268,2811,2863,4034,2689,2003,2284,2735,2912,1994,3261,4723,2091,3094
1,2022.03.21,437,278,575,833,1819,1037,1250,1406,761,1201,3569,2640,1292,1237,2005,2012,1022,560,2486,1341,1694,2064,2834,2335,2378,2012
2,2022.03.20,1023,998,1370,2577,2334,2017,2396,3052,1854,2007,4210,2487,1801,2420,2730,4125,2841,1585,1922,2715,3433,2627,2198,4524,3090,3534
3,2022.03.19,1061,912,1740,2274,2655,2695,2930,3569,2027,2228,4846,3606,2177,2734,3689,3985,3356,1914,4661,3209,4678,2930,5317,4816,2955,5168
4,2022.03.18,1172,977,2045,2649,2982,3197,2820,3751,2248,2565,2898,3556,3283,2706,3722,6367,3122,1950,3703,3020,3066,2802,3892,5107,3628,4769


In [ ]:
def change_time(x):
    if len(x) < 10:
        x = '20' + x
        return x
    else:
        return x

covid_df['date'] = covid_df['date'].map(change_time)
covid_df.head()

,date,종로구,중구,용산구,성동구,광진구,동대문구,중랑구,성북구,강북구,도봉구,노원구,은평구,서대문구,마포구,양천구,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구,기타
0,2022.03.22,1021,1094,1536,2231,1734,2814,3058,2176,2263,2187,3155,2002,2268,2811,2863,4034,2689,2003,2284,2735,2912,1994,3261,4723,2091,3094
1,2022.03.21,437,278,575,833,1819,1037,1250,1406,761,1201,3569,2640,1292,1237,2005,2012,1022,560,2486,1341,1694,2064,2834,2335,2378,2012
2,2022.03.20,1023,998,1370,2577,2334,2017,2396,3052,1854,2007,4210,2487,1801,2420,2730,4125,2841,1585,1922,2715,3433,2627,2198,4524,3090,3534
3,2022.03.19,1061,912,1740,2274,2655,2695,2930,3569,2027,2228,4846,3606,2177,2734,3689,3985,3356,1914,4661,3209,4678,2930,5317,4816,2955,5168
4,2022.03.18,1172,977,2045,2649,2982,3197,2820,3751,2248,2565,2898,3556,3283,2706,3722,6367,3122,1950,3703,3020,3066,2802,3892,5107,3628,4769


In [ ]:
covid_df.tail()

,date,종로구,중구,용산구,성동구,광진구,동대문구,중랑구,성북구,강북구,도봉구,노원구,은평구,서대문구,마포구,양천구,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구,기타
773,2020.02.09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
774,2020.02.08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
775,2020.02.07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
776,2020.02.06,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
777,2020.02.05,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
print(covid_df.columns.values.shape)

(27,)


In [ ]:
covid_df.drop('기타', axis=1, inplace=True)
covid_df['date'] = pd.to_datetime(covid_df['date'])
covid_df.reset_index(drop=True, inplace=True)

In [ ]:
print(covid_df.shape)
covid_df.drop_duplicates(subset=['date'], inplace=True)
print(covid_df.shape)
covid_df.head()

(778, 26)
(777, 26)


,date,종로구,중구,용산구,성동구,광진구,동대문구,중랑구,성북구,강북구,도봉구,노원구,은평구,서대문구,마포구,양천구,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구
0,2022-03-22,1021,1094,1536,2231,1734,2814,3058,2176,2263,2187,3155,2002,2268,2811,2863,4034,2689,2003,2284,2735,2912,1994,3261,4723,2091
1,2022-03-21,437,278,575,833,1819,1037,1250,1406,761,1201,3569,2640,1292,1237,2005,2012,1022,560,2486,1341,1694,2064,2834,2335,2378
2,2022-03-20,1023,998,1370,2577,2334,2017,2396,3052,1854,2007,4210,2487,1801,2420,2730,4125,2841,1585,1922,2715,3433,2627,2198,4524,3090
3,2022-03-19,1061,912,1740,2274,2655,2695,2930,3569,2027,2228,4846,3606,2177,2734,3689,3985,3356,1914,4661,3209,4678,2930,5317,4816,2955
4,2022-03-18,1172,977,2045,2649,2982,3197,2820,3751,2248,2565,2898,3556,3283,2706,3722,6367,3122,1950,3703,3020,3066,2802,3892,5107,3628


In [ ]:
print(covid_df.columns)

Index(['date', '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구',
       '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'],
      dtype='object')


In [ ]:
new_covid = pd.DataFrame(columns=['date', 'district', 'cases'])
# print(covid_df.columns)
for i in range(covid_df.shape[0]):
    date = covid_df['date'].iloc[i]
    for col in covid_df.columns.values[1:]:
        new_covid = new_covid.append({'date': date, 'district': col, 
                                      'cases': covid_df[col].iloc[i]}, ignore_index=True)

new_covid.head()

,date,district,cases
0,2022-03-22,종로구,1021
1,2022-03-22,중구,1094
2,2022-03-22,용산구,1536
3,2022-03-22,성동구,2231
4,2022-03-22,광진구,1734


In [ ]:
print(covid_df.shape)
print(new_covid.shape)

(777, 26)
(19425, 3)


In [ ]:
777 * 25

19425

In [ ]:
new_covid.to_csv(f"{PATH}/covid_cases.csv", encoding='utf8', index=False)

# 상권 영역

In [ ]:
os.listdir()

['서울특별시 코로나19 자치구별 확진자 발생동향.csv',
 '서울시 우리마을가게 상권분석서비스(상권-생활인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권_상주인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권-상권변화지표).csv',
 '서울시 우리마을가게 상권분석서비스(상권-직장인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권-아파트).csv',
 '서울시 우리마을가게 상권분석서비스(상권-추정매출).csv',
 '서울시 우리마을가게 상권분석서비스(상권-점포).csv',
 '서울시 우리마을가게 상권분석서비스(상권영역).csv',
 '서울특별시 건축물대장 공통 새주소코드 정보.csv',
 'seoul_dong.csv',
 'seoul_dong.txt',
 '__MACOSX',
 'sangju.csv',
 'working_pop.csv',
 'calendar.csv',
 'covid_cases.csv',
 'md_region.csv',
 'change.csv',
 'gender_sales.csv',
 'weekday_sales.csv',
 'admin.csv',
 'services.csv',
 'change_codes.csv',
 'md_class_code.csv',
 'unique_md.csv']

In [ ]:
region = pd.read_csv(os.listdir()[8], encoding='cp949')
region.head()

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
0,201810,A,골목상권,1000275,보문로26길,201886,454183,11290,11290600,NaN
1,201810,A,골목상권,1000276,보문로31길,201385,454290,11290,11290555,NaN
2,201810,D,발달상권,1001045,동작구 총신대입구역_1,198304,442948,11590,11590630,NaN
3,201810,D,발달상권,1001046,서울 관악구 신림역_4,193828,443001,11620,11620695,NaN
4,201810,D,발달상권,1001047,뱅뱅사거리_1,202960,443242,11680,11680655,NaN


In [ ]:
region.drop('형태정보', axis=1, inplace=True)
region.columns = ['year_month', 'md_class_code', 'md_class_name', 'mdcode', 'mdname', 'x_coord', 'y_coord', 'district_code', 'hang_code']

## modify time series
ym = region['year_month']
region.drop('year_month', axis=1, inplace=True)
region['year'] = np.zeros(shape=(region.shape[0], ))
region['month'] = np.zeros(shape=(region.shape[0], ))

for i in range(ym.shape[0]):
    row = str(ym.iloc[i])
    year, month = int(row[:4]), int(row[4:])
    region['year'].iloc[i] = year
    region['month'].iloc[i] = month

region = region[['year', 'month', 'md_class_code', 'md_class_name', 'mdcode', 'mdname', 'x_coord', 'y_coord', 'district_code', 'hang_code']]
region.head()

,year,month,md_class_code,md_class_name,mdcode,mdname,x_coord,y_coord,district_code,hang_code
0,2018.0,10.0,A,골목상권,1000275,보문로26길,201886,454183,11290,11290600
1,2018.0,10.0,A,골목상권,1000276,보문로31길,201385,454290,11290,11290555
2,2018.0,10.0,D,발달상권,1001045,동작구 총신대입구역_1,198304,442948,11590,11590630
3,2018.0,10.0,D,발달상권,1001046,서울 관악구 신림역_4,193828,443001,11620,11620695
4,2018.0,10.0,D,발달상권,1001047,뱅뱅사거리_1,202960,443242,11680,11680655


In [ ]:
## modify coordinate
proj1 = pyproj.Proj(init='epsg:5181')
proj2 = pyproj.Proj(init='epsg:4326')
for i in range(region.shape[0]):
    x, y = region['x_coord'].iloc[i], region['y_coord'].iloc[i]
    lon, lat = pyproj.transform(proj1, proj2, x, y)
    region['x_coord'].iloc[i] = lat
    region['y_coord'].iloc[i] = lon

region.head()

,year,month,md_class_code,md_class_name,mdcode,mdname,x_coord,y_coord,district_code,hang_code
0,2018.0,10.0,A,골목상권,1000275,보문로26길,37.587205,127.021354,11290,11290600
1,2018.0,10.0,A,골목상권,1000276,보문로31길,37.588170,127.015681,11290,11290555
2,2018.0,10.0,D,발달상권,1001045,동작구 총신대입구역_1,37.485978,126.980824,11590,11590630
3,2018.0,10.0,D,발달상권,1001046,서울 관악구 신림역_4,37.486436,126.930214,11620,11620695
4,2018.0,10.0,D,발달상권,1001047,뱅뱅사거리_1,37.488623,127.033469,11680,11680655


In [ ]:
region.to_csv(f"{PATH}/md_region.csv", encoding='utf8', index=False)

# 상권 변화지표

In [ ]:
os.listdir()

['서울특별시 코로나19 자치구별 확진자 발생동향.csv',
 '서울시 우리마을가게 상권분석서비스(상권-생활인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권_상주인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권-상권변화지표).csv',
 '서울시 우리마을가게 상권분석서비스(상권-직장인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권-아파트).csv',
 '서울시 우리마을가게 상권분석서비스(상권-추정매출).csv',
 '서울시 우리마을가게 상권분석서비스(상권-점포).csv',
 '서울시 우리마을가게 상권분석서비스(상권영역).csv',
 '서울특별시 건축물대장 공통 새주소코드 정보.csv',
 'seoul_dong.csv',
 'seoul_dong.txt',
 '__MACOSX',
 'sangju.csv',
 'working_pop.csv',
 'calendar.csv',
 'covid_cases.csv',
 'md_region.csv',
 'change.csv',
 'gender_sales.csv',
 'weekday_sales.csv',
 'admin.csv',
 'services.csv',
 'change_codes.csv',
 'md_class_code.csv',
 'unique_md.csv']

In [ ]:
change = pd.read_csv(os.listdir()[3], encoding='cp949', engine='python')
change.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2021,4,U,관광특구,1001496,강남 마이스 관광특구,HH,정체,136,64,118,55
1,2021,4,U,관광특구,1001495,잠실 관광특구,LL,다이나믹,113,54,118,55
2,2021,4,U,관광특구,1001494,종로?청계 관광특구,HH,정체,158,74,118,55
3,2021,4,U,관광특구,1001493,동대문패션타운 관광특구,HH,정체,140,63,118,55
4,2021,4,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,HH,정체,163,65,118,55


In [ ]:
change.columns = ['year', 'quarter', 'md_class_code', 'md_class_name', 'md_code', 'md_code_name',
                  'change_index', 'change_name', 'open_month', 'closed_month', 'seoul_open_month', 'seoul_closed_month']
change.head()

,year,quarter,md_class_code,md_class_name,md_code,md_code_name,change_index,change_name,open_month,closed_month,seoul_open_month,seoul_closed_month
0,2021,4,U,관광특구,1001496,강남 마이스 관광특구,HH,정체,136,64,118,55
1,2021,4,U,관광특구,1001495,잠실 관광특구,LL,다이나믹,113,54,118,55
2,2021,4,U,관광특구,1001494,종로?청계 관광특구,HH,정체,158,74,118,55
3,2021,4,U,관광특구,1001493,동대문패션타운 관광특구,HH,정체,140,63,118,55
4,2021,4,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,HH,정체,163,65,118,55


In [ ]:
change.to_csv(f"{PATH}/change.csv", encoding='utf8', index=False)

# 추정매출

In [ ]:
os.listdir()

['서울특별시 코로나19 자치구별 확진자 발생동향.csv',
 '서울시 우리마을가게 상권분석서비스(상권-생활인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권_상주인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권-상권변화지표).csv',
 '서울시 우리마을가게 상권분석서비스(상권-직장인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권-아파트).csv',
 '서울시 우리마을가게 상권분석서비스(상권-추정매출).csv',
 '서울시 우리마을가게 상권분석서비스(상권-점포).csv',
 '서울시 우리마을가게 상권분석서비스(상권영역).csv',
 '서울특별시 건축물대장 공통 새주소코드 정보.csv',
 'seoul_dong.csv',
 'seoul_dong.txt',
 '__MACOSX',
 'sangju.csv',
 'change.csv',
 'working_pop.csv',
 'admin.csv',
 'covid_cases.csv',
 'md_region.csv',
 'services.csv',
 'unique_md.csv',
 'md_class_code.csv',
 'change_codes.csv',
 'calendar.csv',
 'gender_sales.csv',
 'weekday_sales.csv',
 'md_info.gsheet']

In [ ]:
sale = pd.read_csv(os.listdir()[6], encoding='cp949', engine='python')
sale.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,시간대_00~06_매출_비율,시간대_06~11_매출_비율,시간대_11~14_매출_비율,시간대_14~17_매출_비율,시간대_17~21_매출_비율,시간대_21~24_매출_비율,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,연령대_40_매출_비율,연령대_50_매출_비율,연령대_60_이상_매출_비율,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,시간대_00~06_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,주중_매출_건수,주말_매출_건수,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,시간대_건수~06_매출_건수,시간대_건수~11_매출_건수,시간대_건수~14_매출_건수,시간대_건수~17_매출_건수,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,10000000,94,55,45,0,0,0,24,31,20,25,0,4,15,41,41,0,36,65,0,1,66,14,19,0,5514914,4485086,0,0,0,2370401,3144513,1964692,2520394,0,349012,1544253,4051234,4055501,0,3553183,6446817,0,137299,6562539,1366053,1934109,0,43,51,0,0,0,12,31,18,33,0,9,22,41,22,0,33,61,0,9,54,14,17,0,8
1,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300035,인테리어,12566919,370,100,0,0,0,100,0,0,0,0,0,0,0,0,100,0,0,100,0,0,0,0,100,0,12566919,0,0,0,12566919,0,0,0,0,0,0,0,0,12566919,0,0,12566919,0,0,0,0,12566919,0,370,0,0,0,370,0,0,0,0,0,0,0,0,370,0,0,370,0,0,0,0,370,0,4
2,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300032,가전제품,1717076948,8760,65,35,13,10,7,14,21,22,14,0,1,29,27,43,0,57,44,1,19,23,21,17,20,1114559027,602517921,214008061,178582703,126144410,240632078,355191775,371001441,231516480,0,18672572,490337382,458946235,743156581,5964178,860083470,663335187,16795861,290757468,348920397,318762634,251461642,296720655,6040,2720,1128,1217,810,1052,1833,1640,1080,0,126,2507,2763,3298,66,3585,4551,144,1800,1852,1392,926,2021,8
3,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300031,가구,11911789492,6524,55,45,12,7,11,12,12,30,15,0,6,26,48,21,0,44,56,0,0,9,20,30,41,6570250285,5341539207,1472780295,807662007,1345887821,1469403792,1474516370,3527967301,1813571906,0,699719298,3033809617,5658056190,2520204387,0,5268497054,6643292438,0,4707327,1073648650,2370848619,3552799560,4909785336,3676,2848,611,623,798,833,811,1943,905,0,305,1834,2857,1528,0,2704,3820,0,23,494,1493,1634,2880,6
4,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300028,화초,997157377,14124,84,16,22,9,15,20,17,9,8,0,3,24,24,48,1,45,55,0,10,31,34,15,10,833148687,164008690,223344287,91322206,148447228,201915183,168119783,86244326,77764364,0,29328827,237789810,236439189,479876520,13723031,347053856,418682753,912095,76937405,233213516,262016189,113412321,79245081,9891,4233,1601,1514,1861,2053,2862,2227,2006,0,1097,3967,3290,5347,423,6916,5717,105,3128,4161,2715,1194,1328,5


In [ ]:
sale['기준_년_코드'].unique()

array([2021])

In [ ]:
gender_col = list(sale.columns.values[:8])
for col in sale.columns.values:
    if ('남성' in col or '여성' in col):
        if '금액' in col:
            gender_col.append(col)

gender_col

['기준_년_코드',
 '기준_분기_코드',
 '상권_구분_코드',
 '상권_구분_코드_명',
 '상권_코드',
 '상권_코드_명',
 '서비스_업종_코드',
 '서비스_업종_코드_명',
 '남성_매출_금액',
 '여성_매출_금액']

In [ ]:
weekday_col = list(sale.columns.values[:8])
for col in sale.columns.values:
    if ('월요일' in col or '화요일' in col or '수요일' in col or '목요일' in col or '금요일' in col or '토요일' in col or '일요일' in col):
        if '금액' in col:
            weekday_col.append(col)

weekday_col

['기준_년_코드',
 '기준_분기_코드',
 '상권_구분_코드',
 '상권_구분_코드_명',
 '상권_코드',
 '상권_코드_명',
 '서비스_업종_코드',
 '서비스_업종_코드_명',
 '월요일_매출_금액',
 '화요일_매출_금액',
 '수요일_매출_금액',
 '목요일_매출_금액',
 '금요일_매출_금액',
 '토요일_매출_금액',
 '일요일_매출_금액']

In [ ]:
gender_sales = sale[gender_col]
weekday_sales = sale[weekday_col]

In [ ]:
gender_sales.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,남성_매출_금액,여성_매출_금액
0,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,3553183,6446817
1,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300035,인테리어,0,12566919
2,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300032,가전제품,860083470,663335187
3,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300031,가구,5268497054,6643292438
4,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300028,화초,347053856,418682753


In [ ]:
weekday_sales.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액
0,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,0,0,0,2370401,3144513,1964692,2520394
1,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300035,인테리어,0,0,12566919,0,0,0,0
2,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300032,가전제품,214008061,178582703,126144410,240632078,355191775,371001441,231516480
3,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300031,가구,1472780295,807662007,1345887821,1469403792,1474516370,3527967301,1813571906
4,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300028,화초,223344287,91322206,148447228,201915183,168119783,86244326,77764364


In [ ]:
gender_sales.columns = ['year', 'quarter', 'md_class_code', 'md_class_name', 'md_code', 'md_code_name',
                        'dept_code', 'dept_name', 'male_sales', 'female_sales']
weekday_sales.columns = ['year', 'quarter', 'md_class_code', 'md_class_name', 'md_code', 'md_code_name', 
                         'dept_code', 'dept_name', 'mon_sales', 'tue_sales', 'wed_sales', 'thu_sales', 'fri_sales', 'sat_sales', 'sun_sales']

In [ ]:
gender_sales.head()

,year,quarter,md_class_code,md_class_name,md_code,md_code_name,dept_code,dept_name,male_sales,female_sales
0,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,3553183,6446817
1,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300035,인테리어,0,12566919
2,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300032,가전제품,860083470,663335187
3,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300031,가구,5268497054,6643292438
4,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300028,화초,347053856,418682753


In [ ]:
gender_sales.drop(['md_class_name', 'md_code_name', 'dept_name'], axis=1, inplace=True)
weekday_sales.drop(['md_class_name', 'md_code_name', 'dept_name'], axis=1, inplace=True)

In [ ]:
gencol = ['male_sales', 'female_sales']
new_gender_sales = pd.DataFrame()
for i in range(gender_sales.shape[0]):
    year, quarter = gender_sales['year'].iloc[i], gender_sales['quarter'].iloc[i]
    class_code, md_code, dept_code = gender_sales['md_class_code'].iloc[i], gender_sales['md_code'].iloc[i], gender_sales['dept_code'].iloc[i]
    for j in range(len(gencol)):
        if j == 0:
            gender = "남성"
        else:
            gender = "여성"
        col = gencol[j]
        new_gender_sales = new_gender_sales.append({'year': year, 'quarter': quarter,
                                            'md_class_code': class_code,
                                            'md_code': md_code, 'dept_code': dept_code,
                                            'gender': gender, 'sales': gender_sales[col].iloc[i]}, 
                                            ignore_index=True)

In [ ]:
# new_gender_sales.head()

,year,quarter,md_class_code,md_code,dept_code,gender,sales
0,2021.0,4.0,U,1001496.0,CS300043,남성,3553183.0
1,2021.0,4.0,U,1001496.0,CS300043,여성,6446817.0
2,2021.0,4.0,U,1001496.0,CS300035,남성,0.0
3,2021.0,4.0,U,1001496.0,CS300035,여성,12566919.0
4,2021.0,4.0,U,1001496.0,CS300032,남성,860083470.0


In [ ]:
# week_cols = ['mon_sales', 'tue_sales', 'wed_sales', 'thu_sales', 'fri_sales', 'sat_sales', 'sun_sales']

# new_weekday_sales = pd.DataFrame()
# for i in range(weekday_sales.shape[0]):
#     year, quarter = weekday_sales['year'].iloc[i], weekday_sales['quarter'].iloc[i]
#     class_code, md_code, dept_code = weekday_sales['md_class_code'].iloc[i], weekday_sales['md_code'].iloc[i], weekday_sales['dept_code'].iloc[i]
#     for j in range(len(week_cols)):
#         col = week_cols[j]
#         new_weekday_sales = new_weekday_sales.append({'year': year, 'quarter': quarter,
#                                                       'md_class_code': class_code, 
#                                                       'md_code': md_code, 'dept_code': dept_code,
#                                                       'weekday': j, 'sales': weekday_sales[col].iloc[i]},
#                                                      ignore_index=True)

In [ ]:
# new_weekday_sales.head()

,year,quarter,md_class_code,md_code,dept_code,weekday,sales
0,2021.0,4.0,U,1001496.0,CS300043,0.0,0.0
1,2021.0,4.0,U,1001496.0,CS300043,1.0,0.0
2,2021.0,4.0,U,1001496.0,CS300043,2.0,0.0
3,2021.0,4.0,U,1001496.0,CS300043,3.0,2370401.0
4,2021.0,4.0,U,1001496.0,CS300043,4.0,3144513.0


In [ ]:
new_gender_sales.to_csv(f"nd{PATH}/geer_sales.csv", encoding='utf-8', index=False)
# weekday_sales.to_csv(f"{PATH}/weekday_sales.csv", encoding='utf-8', index=False)

# 마지막 정리

In [ ]:
os.listdir()

['서울특별시 코로나19 자치구별 확진자 발생동향.csv',
 '서울시 우리마을가게 상권분석서비스(상권-생활인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권_상주인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권-상권변화지표).csv',
 '서울시 우리마을가게 상권분석서비스(상권-직장인구).csv',
 '서울시 우리마을가게 상권분석서비스(상권-아파트).csv',
 '서울시 우리마을가게 상권분석서비스(상권-추정매출).csv',
 '서울시 우리마을가게 상권분석서비스(상권-점포).csv',
 '서울시 우리마을가게 상권분석서비스(상권영역).csv',
 '서울특별시 건축물대장 공통 새주소코드 정보.csv',
 'seoul_dong.csv',
 'seoul_dong.txt',
 '__MACOSX',
 'sangju.csv',
 'change.csv',
 'working_pop.csv',
 'admin.csv',
 'covid_cases.csv',
 'md_region.csv',
 'services.csv',
 'unique_md.csv',
 'md_class_code.csv',
 'change_codes.csv',
 'calendar.csv',
 'gender_sales.csv',
 'weekday_sales.csv']

In [ ]:
macos_idx = os.listdir().index('__MACOSX')
later = os.listdir()[macos_idx+1:]
later

['sangju.csv',
 'change.csv',
 'working_pop.csv',
 'admin.csv',
 'covid_cases.csv',
 'md_region.csv',
 'services.csv',
 'unique_md.csv',
 'md_class_code.csv',
 'change_codes.csv',
 'calendar.csv',
 'gender_sales.csv',
 'weekday_sales.csv']

In [ ]:
for f in later:
    df = pd.read_csv(f"{PATH}/{f}")
    columns = list(df.columns.values)
    if 'mdcode' in columns:
        idx = columns.index('mdcode')
        df.columns.values[idx] = 'md_code'
    if 'md_class_code_name' in columns:
        idx = columns.index('md_class_code_name')
        df.columns.values[idx] = 'md_class_name'
    if 'hang_code' in columns:
        idx = columns.index('hang_code')
        df.columns.values[idx] = 'admin_code'
    if 'code' in columns:
        idx = columns.index('code')
        df.columns.values[idx] = 'admin_code'
    print(df.columns.values)
    df.to_csv(f"{PATH}/{f}", encoding='utf-8', index=False)

['year' 'quarter' 'md_code' 'gender' 'age' 'population']
['year' 'quarter' 'md_class_code' 'md_class_name' 'md_code' 'md_code_name'
 'change_index' 'change_name' 'open_month' 'closed_month'
 'seoul_open_month' 'seoul_closed_month']
['year' 'quarter' 'md_code' 'gender' 'age' 'population']
['district_code' 'district' 'admin_code' 'admin' 'latitude' 'longitude']
['date' 'district' 'cases']
['year' 'month' 'md_class_code' 'md_class_name' 'md_code' 'mdname'
 'x_coord' 'y_coord' 'district_code' 'admin_code']
['dept_code' 'dept_name']
['md_code' 'md_code_name' 'md_class_code' 'change_index']
['md_class_code' 'md_class_name']
['change_index' 'change_name']
['date' 'year' 'month' 'day' 'weekday' 'quarter' 'season' 'legal_holiday'
 'agg_holiday']
['year' 'quarter' 'md_class_code' 'md_code' 'dept_code' 'gender' 'sales']
['year' 'quarter' 'md_class_code' 'md_code' 'dept_code' 'weekday' 'sales']


In [ ]:
admin = pd.read_csv(f"{PATH}/admin.csv", engine='python')
region = pd.read_csv(f"{PATH}/md_region.csv", engine='python')

In [ ]:
admin.head()

,district_code,district,admin_code,admin,latitude,longitude
0,11110,종로구,11110515,청운효자동,37.584009,126.970626
1,11110,종로구,11110530,사직동,37.575408,126.965944
2,11110,종로구,11110540,삼청동,37.590758,126.980996
3,11110,종로구,11110550,부암동,37.594768,126.965574
4,11110,종로구,11110560,평창동,37.613029,126.974485


In [ ]:
region.head()

,year,month,md_class_code,md_class_name,md_code,mdname,x_coord,y_coord,district_code,admin_code
0,2018.0,10.0,A,골목상권,1000275,보문로26길,37.587205,127.021354,11290,11290600
1,2018.0,10.0,A,골목상권,1000276,보문로31길,37.588170,127.015681,11290,11290555
2,2018.0,10.0,D,발달상권,1001045,동작구 총신대입구역_1,37.485978,126.980824,11590,11590630
3,2018.0,10.0,D,발달상권,1001046,서울 관악구 신림역_4,37.486436,126.930214,11620,11620695
4,2018.0,10.0,D,발달상권,1001047,뱅뱅사거리_1,37.488623,127.033469,11680,11680655


In [ ]:
admin_codes = admin[['admin_code', 'admin', 'district']]
region_codes = region[['district_code', 'admin_code']]
m = pd.merge(left=admin_codes, right=region_codes, on='admin_code', how='left')

In [ ]:
m.isnull().sum()

admin_code        0
admin             0
district          0
district_code    55
dtype: int64

In [ ]:
mnull = m.isnull()
trueidx = mnull[mnull['district_code'] == False].index
trues = m.loc[trueidx].reset_index(drop=True)

code_dist = dict()
for i in range(trues.shape[0]):
    code, name = trues['district_code'].iloc[i], trues['district'].iloc[i]
    if code not in code_dist:
        code_dist[name] = int(code)

print(len(code_dist))
code_dist

25


{'강남구': 11680,
 '강동구': 11740,
 '강북구': 11305,
 '강서구': 11500,
 '관악구': 11620,
 '광진구': 11215,
 '구로구': 11530,
 '금천구': 11545,
 '노원구': 11350,
 '도봉구': 11320,
 '동대문구': 11230,
 '동작구': 11590,
 '마포구': 11440,
 '서대문구': 11410,
 '서초구': 11650,
 '성동구': 11200,
 '성북구': 11290,
 '송파구': 11710,
 '양천구': 11470,
 '영등포구': 11560,
 '용산구': 11170,
 '은평구': 11380,
 '종로구': 11110,
 '중구': 11140,
 '중랑구': 11260}

In [ ]:
admin['district_code'] = admin['district'].map(code_dist)
admin = admin[['district_code', 'district', 'admin_code', 'admin', 'latitude', 'longitude']]
admin.head()

,district_code,district,admin_code,admin,latitude,longitude
0,11110,종로구,11110515,청운효자동,37.584009,126.970626
1,11110,종로구,11110530,사직동,37.575408,126.965944
2,11110,종로구,11110540,삼청동,37.590758,126.980996
3,11110,종로구,11110550,부암동,37.594768,126.965574
4,11110,종로구,11110560,평창동,37.613029,126.974485


In [ ]:
admin.isnull().sum()

district_code    0
district         0
admin_code       0
admin            0
latitude         0
longitude        0
dtype: int64

In [ ]:
cases = pd.read_csv(f"{PATH}/covid_cases.csv", engine='python')
cases['district_code'] = cases['district'].map(code_dist)
cases.head()

,date,district,cases,district_code
0,2022-03-22,종로구,1021,11110
1,2022-03-22,중구,1094,11140
2,2022-03-22,용산구,1536,11170
3,2022-03-22,성동구,2231,11200
4,2022-03-22,광진구,1734,11215


In [ ]:
import datetime

for i in range(cases.shape[0]):
    dt = cases['date'].iloc[i]
    dt = dt.split(' ')[0]
    year, month, day = dt.split('-')
    dt_form = datetime.date(int(year), int(month), int(day))
    cases['date'].iloc[i] = dt_form

In [ ]:
cases['date'] = pd.to_datetime(cases['date'])
cases['district_code'] = cases['district_code'].astype(np.int64)
cases = cases[['date', 'district_code', 'district', 'cases']]

In [ ]:
cases.head()

,date,district_code,district,cases
0,2022-03-22,11110,종로구,1021
1,2022-03-22,11140,중구,1094
2,2022-03-22,11170,용산구,1536
3,2022-03-22,11200,성동구,2231
4,2022-03-22,11215,광진구,1734


In [ ]:
print(new_covid.shape)
print(cases.shape)

(19425, 3)
(19425, 4)


In [ ]:
## group by count
cnt = cases.groupby(by=['date', 'district']).count()
cnt.reset_index(drop=False, inplace=True)
cnt.head()

,date,district,district_code,cases
0,2020-02-05,강남구,1,1
1,2020-02-05,강동구,1,1
2,2020-02-05,강북구,1,1
3,2020-02-05,강서구,1,1
4,2020-02-05,관악구,1,1


In [ ]:
cnt[cnt['cases'] > 1]

,date,district,district_code,cases


In [ ]:
# admin.to_csv(f"{PATH}/admin.csv", encoding='utf-8', index=False)
# region.to_csv(f"{PATH}/md_region.csv", encoding='utf-8', index=False)
cases.to_csv(f"{PATH}/covid_cases.csv", encoding='utf-8', index=False)

# Table 정규화

## service table

In [ ]:
weekday_sales.head()

,year,quarter,md_class_code,md_class_name,md_code,md_code_name,dept_code,dept_name,mon_sales,tue_sales,wed_sales,thu_sales,fri_sales,sat_sales,sun_sales
0,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,0,0,0,2370401,3144513,1964692,2520394
1,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300035,인테리어,0,0,12566919,0,0,0,0
2,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300032,가전제품,214008061,178582703,126144410,240632078,355191775,371001441,231516480
3,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300031,가구,1472780295,807662007,1345887821,1469403792,1474516370,3527967301,1813571906
4,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300028,화초,223344287,91322206,148447228,201915183,168119783,86244326,77764364


In [ ]:
gender_sales.head()

,year,quarter,md_class_code,md_class_name,md_code,md_code_name,dept_code,dept_name,male_sales,female_sales
0,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300043,전자상거래업,3553183,6446817
1,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300035,인테리어,0,12566919
2,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300032,가전제품,860083470,663335187
3,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300031,가구,5268497054,6643292438
4,2021,4,U,관광특구,1001496,강남 마이스 관광특구,CS300028,화초,347053856,418682753


In [ ]:
sales = pd.concat([weekday_sales[['dept_code', 'dept_name']], gender_sales[['dept_code', 'dept_name']]], axis=0)
sales.reset_index(drop=True, inplace=True)

codes, names = [], []
for i in range(sales.shape[0]):
    code = sales['dept_code'].iloc[i]
    if code not in codes:
        name = sales['dept_name'].iloc[i]
        codes.append(code)
        names.append(name)

print(len(codes) == len(names))

True


In [ ]:
services = pd.DataFrame()
services['dept_code'] = codes
services['dept_name'] = names
print(services.shape)
services.head()

(63, 2)


,dept_code,dept_name
0,CS300043,전자상거래업
1,CS300035,인테리어
2,CS300032,가전제품
3,CS300031,가구
4,CS300028,화초


In [ ]:
services.to_csv(f"{PATH}/services.csv", encoding='utf-8', index=False)

## 상권 변화 지표 코드 - 상권 변화 지표 명

In [ ]:
change.head()

,year,quarter,md_class_code,md_class_name,md_code,md_code_name,change_index,change_name,open_month,closed_month,seoul_open_month,seoul_closed_month
0,2021,4,U,관광특구,1001496,강남 마이스 관광특구,HH,정체,136,64,118,55
1,2021,4,U,관광특구,1001495,잠실 관광특구,LL,다이나믹,113,54,118,55
2,2021,4,U,관광특구,1001494,종로?청계 관광특구,HH,정체,158,74,118,55
3,2021,4,U,관광특구,1001493,동대문패션타운 관광특구,HH,정체,140,63,118,55
4,2021,4,U,관광특구,1001492,명동 남대문 북창동 다동 무교동 관광특구,HH,정체,163,65,118,55


In [ ]:
print(change['change_index'].unique())
print(change['change_name'].unique())

['HH' 'LL' 'HL' 'LH']
['정체' '다이나믹' '상권축소' '상권확장']


In [ ]:
change_codes = pd.DataFrame(columns=['change_index', 'change_name'])
change_codes['change_index'] = change['change_index'].unique()
change_codes['change_name'] = change['change_name'].unique()
change_codes

,change_index,change_name
0,HH,정체
1,LL,다이나믹
2,HL,상권축소
3,LH,상권확장


In [ ]:
change_codes.to_csv(f"{PATH}/change_codes.csv", encoding='utf-8', index=False)

## 상권 코드

In [ ]:
region.head()

,year,month,md_class_code,md_class_name,md_code,mdname,x_coord,y_coord,district_code,admin_code
0,2018.0,10.0,A,골목상권,1000275,보문로26길,37.587205,127.021354,11290,11290600
1,2018.0,10.0,A,골목상권,1000276,보문로31길,37.588170,127.015681,11290,11290555
2,2018.0,10.0,D,발달상권,1001045,동작구 총신대입구역_1,37.485978,126.980824,11590,11590630
3,2018.0,10.0,D,발달상권,1001046,서울 관악구 신림역_4,37.486436,126.930214,11620,11620695
4,2018.0,10.0,D,발달상권,1001047,뱅뱅사거리_1,37.488623,127.033469,11680,11680655


In [ ]:
regcode = region[['md_code', 'mdname', 'md_class_code', 'md_class_name']]
regcode.columns = ['md_code', 'md_code_name', 'md_class_code', 'md_class_name']

mdcodes = pd.concat([regcode, weekday_sales[['md_code', 'md_code_name', 'md_class_code', 'md_class_name']], 
                     gender_sales[['md_code', 'md_code_name', 'md_class_code', 'md_class_name']], 
                     change[['md_code', 'md_code_name', 'md_class_code', 'md_class_name']]], 
                    axis=0)
mdcodes.reset_index(drop=True, inplace=True)
print(mdcodes.shape)
mdcodes.head()

(306774, 4)


,md_code,md_code_name,md_class_code,md_class_name
0,1000275,보문로26길,A,골목상권
1,1000276,보문로31길,A,골목상권
2,1001045,동작구 총신대입구역_1,D,발달상권
3,1001046,서울 관악구 신림역_4,D,발달상권
4,1001047,뱅뱅사거리_1,D,발달상권


In [ ]:
unique_md = pd.DataFrame()

code_unique = []
name_unique = []
class_unique = []
cname_unique = []
for i in range(regcode.shape[0]):
    name = regcode['md_code_name'].iloc[i]
    if name not in name_unique:
        code = regcode['md_code'].iloc[i]
        class_ = regcode['md_class_code'].iloc[i]
        cname = regcode['md_class_name'].iloc[i]
        name_unique.append(name)
        code_unique.append(code)
        class_unique.append(class_)

unique_md['md_code'] = code_unique
unique_md['md_code_name'] = name_unique
unique_md['md_class_code'] = class_unique

unique_md

,md_code,md_code_name,md_class_code
0,1000275,보문로26길,A
1,1000276,보문로31길,A
2,1001045,동작구 총신대입구역_1,D
3,1001046,서울 관악구 신림역_4,D
4,1001047,뱅뱅사거리_1,D
...,...,...,...
1491,1001153,당산역_1,D
1492,1001154,용산 전자상가_4,D
1493,1001155,용산 전자상가_5,D
1494,1001156,서울 강동구 길동역,D


In [ ]:
unique_class = pd.DataFrame()

class_unique = []
cname_unique = []
for i in range(regcode.shape[0]):
    class_ = regcode['md_class_code'].iloc[i]
    if class_ not in class_unique:
        cname = regcode['md_class_name'].iloc[i]
        cname_unique.append(cname)
        class_unique.append(class_)

unique_class['md_class_code'] = class_unique
unique_class['md_class_name'] = cname_unique

unique_class

,md_class_code,md_class_name
0,A,골목상권
1,D,발달상권
2,R,전통시장
3,U,관광특구


In [ ]:
unique_change = pd.DataFrame()

code_unique = []
change_unique = []
for i in range(change.shape[0]):
    code = change['md_code'].iloc[i]
    if code not in code_unique:
        chg = change['change_index'].iloc[i]
        code_unique.append(code)
        change_unique.append(chg)

unique_change['md_code'] = code_unique
unique_change['change_index'] = change_unique

unique_change

,md_code,change_index
0,1001496,HH
1,1001495,LL
2,1001494,HH
3,1001493,HH
4,1001492,HH
...,...,...
1491,1000005,LL
1492,1000004,LH
1493,1000003,HH
1494,1000002,HL


In [ ]:
unique_merge = pd.merge(left=unique_md, right=unique_change, on='md_code', how='left')
unique_merge

,md_code,md_code_name,md_class_code,change_index
0,1000275,보문로26길,A,HL
1,1000276,보문로31길,A,HH
2,1001045,동작구 총신대입구역_1,D,HL
3,1001046,서울 관악구 신림역_4,D,LL
4,1001047,뱅뱅사거리_1,D,LL
...,...,...,...,...
1491,1001153,당산역_1,D,LL
1492,1001154,용산 전자상가_4,D,HL
1493,1001155,용산 전자상가_5,D,HH
1494,1001156,서울 강동구 길동역,D,LL


In [ ]:
unique_class.to_csv(f"{PATH}/md_class_code.csv", encoding='utf-8', index=False)
unique_merge.to_csv(f"{PATH}/unique_md.csv", encoding='utf-8', index=False)

In [ ]:
admin = pd.read_csv(f"{PATH}/admin.csv")
mdinfo = read_csv(file_id="1NpuiOR1LcTSC8ebETknhpj5ZQDsO3puO3_10OJ4RKok",
                  is_csv=True, location=f"{PATH}/md_info.gsheet")

In [ ]:
admin.head()

,district_code,district,admin_code,admin,latitude,longitude
0,11110,종로구,11110515,청운효자동,37.584009,126.970626
1,11110,종로구,11110530,사직동,37.575408,126.965944
2,11110,종로구,11110540,삼청동,37.590758,126.980996
3,11110,종로구,11110550,부암동,37.594768,126.965574
4,11110,종로구,11110560,평창동,37.613029,126.974485


In [ ]:
admin[admin['admin_code'] == 11305630]

,district_code,district,admin_code,admin,latitude,longitude


In [ ]:
mdinfo.head()

,admin_code
0,11320511
1,11560670
2,11740580
3,11710631
4,11170685


In [ ]:
admin.shape

(424, 6)

In [ ]:
for code in mdinfo['admin_code']:
    if int(code) not in admin['admin_code']:
        print(code)

11320511
11560670
11740580
11710631
11170685
11740520
11530770
11320514
11710710
11110630
11350560
11560720
11680650
11200560
11590550
11680630
11350630
11110600
11305590
11230610
11680590
11350665
11560680
11140650
11110700
11590530
11545700
11620545
11140635
11650581
11680580
11590540
11470590
11440555
11350595
11710620
11710550
11545630
11170690
11350670
11500540
11545610
11470570
11290810
11530550
11650610
11590660
11590510
11410615
11320680
11470620
11170570
11350624
11200535
11620745
11740700
11470530
11380552
11710610
11110680
11560690
11215740
11710561
11560700
11500620
11620595
11380631
11200650
11530750
11260540
11620525
11545710
11680610
11170650
11620685
11200615
11140590
11500591
11530595
11680531
11170640
11560620
11320522
11230650
11215830
11620605
11110615
11500560
11170580
11440740
11200790
11440600
11560535
11740685
11230570
11560550
11410660
11710642
11290555
11440700
11380510
11500593
11740650
11500603
11290770
11215840
11680730
11590630
11620735
11620630
11410685
1